In [1]:
from lxml import html
import requests
import time
import re
import cssselect

In [2]:
prob_url = "https://www.fangraphs.com/livescoreboard.aspx?date={date}".format(date = time.strftime("%Y-%m-%d"))
prob_page = requests.get(prob_url)
prob_tree = html.fromstring(prob_page.content)

In [3]:
prob_teams = []
for a in prob_tree.xpath('//tr//td//a[contains(@href, "winss.aspx")]'):
    prob_teams.append(re.search(r"team=([a-zA-Z ]*)&", str([a.attrib['href']])).group(1).lower())
prob_odds = prob_tree.xpath('//center/table//tr/td[@align="center"]/text()')

In [5]:
probs_dict = {}
i=0
while i < len(prob_odds):
    probs_dict[prob_teams[i]]=prob_odds[i]
    i+=1
print(probs_dict)

{'orioles': '50.3 %', 'white sox': '49.7 %', 'diamondbacks': '52.6 %', 'brewers': '47.4 %', 'tigers': '38.9 %', 'twins': '61.1 %', 'padres': '34.3 %', 'nationals': '65.7 %', 'yankees': '65.3 %', 'rangers': '34.7 %', 'marlins': '36.9 %', 'mets': '63.1 %', 'braves': '45.9 %', 'phillies': '54.1 %', 'royals': '36.1 %', 'cardinals': '63.9 %', 'rockies': '38.3 %', 'dodgers': '61.7 %'}


In [6]:
betting_page = requests.get('http://www.vegasinsider.com/mlb/odds/las-vegas/')
betting_tree = html.fromstring(betting_page.content)
bet_teams = []
for a in betting_tree.xpath('//td[@class="viBodyBorderNorm"]//td//b/a[contains(@href, "/team/")]'):
    bet_teams.append(re.search(r"team\/([a-zA-Z -]+)", str([a.attrib['href']])).group(1).lower().replace('-', ' '))
bet_odds = betting_tree.xpath('//td[@class="viBodyBorderNorm"]//td[3]//a[contains(@href, "/line-movement/")]/text()')

In [7]:
total_odds = {}
i=0
j=0
while i < len(bet_teams):
    if i%3:
        if bet_teams[j] not in total_odds.keys():
            total_odds[bet_teams[j]]=bet_odds[i][:4]
            j+=1
    i+=1
print(total_odds)

{'padres': '+165', 'nationals': '-180', 'braves': '-101', 'phillies': '-109', 'marlins': '+120', 'mets': '-130', 'diamondbacks': '-115', 'brewers': '+105', 'rockies': '+160', 'dodgers': '-180', 'yankees': '-205', 'rangers': '+185', 'orioles': '-105', 'white sox': '-105', 'tigers': '+185', 'twins': '-210', 'royals': '+180', 'cardinals': '-200'}


In [21]:
class Game:
    "A class holding opponents, odds, probabilities, and any other stat concerning the two teams playing a certain game"
    def __init__(self, team1, team2, team1odds, team2odds, team1winpct, team2winpct):
        self.team1 = team1
        self.team2 = team2
        self.team1odds = team1odds
        self.team2odds = team2odds
        self.team1winpct = team1winpct
        self.team2winpct = team2winpct

In [22]:
i=0
games={}
while i < len(prob_odds):
    key = 'game_'+str(i/2 + 1)
    games[key]=Game(prob_teams[i], prob_teams[i+1], total_odds[prob_teams[i]], total_odds[prob_teams[i+1]], 
                    prob_odds[i], prob_odds[i+1])
    i+=2
print(games)

{'game_1.0': <__main__.Game object at 0x111139550>, 'game_2.0': <__main__.Game object at 0x111139588>, 'game_3.0': <__main__.Game object at 0x1111395c0>, 'game_4.0': <__main__.Game object at 0x1111395f8>, 'game_5.0': <__main__.Game object at 0x111139630>, 'game_6.0': <__main__.Game object at 0x111139668>, 'game_7.0': <__main__.Game object at 0x1111396a0>, 'game_8.0': <__main__.Game object at 0x1111396d8>, 'game_9.0': <__main__.Game object at 0x111139710>}


In [26]:
games['game_1.0'].team1winpct

'50.3 %'